<a href="https://colab.research.google.com/github/SARTHAK4U/GPU-Parallel-Image-Steganography/blob/master/C%2B%2B_Cuda_Steganography.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Setting Up CUDA**

In [ ]:
!apt-get --purge remove cuda nvidia* libnvidia-*
!dpkg -l | grep cuda- | awk '{print $2}' | xargs -n1 dpkg --purge
!apt-get remove cuda-*
!apt autoremove
!apt-get update

In [ ]:
!wget https://developer.nvidia.com/compute/cuda/9.2/Prod/local_installers/cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64 -O cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!dpkg -i cuda-repo-ubuntu1604-9-2-local_9.2.88-1_amd64.deb
!apt-key add /var/cuda-repo-9-2-local/7fa2af80.pub
!apt-get update
!apt-get install cuda-9.2

In [3]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2018 NVIDIA Corporation
Built on Wed_Apr_11_23:16:29_CDT_2018
Cuda compilation tools, release 9.2, V9.2.88


In [ ]:
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

In [5]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


**STEGANOGRAPHY**

**1)Parallel Encryption ->CUDA**

In [6]:
%%cu
#include "/content/ppmHelper.h"
#include "/content/mp3Helper.h"

#include <bits/stdc++.h>

using namespace std;

#define THREADS_PER_BLOCK 512

__global__
void encrypt(unsigned char * inputImageData, unsigned char * outputImageData, int width, int height,
        char * audioData, long long audioSize) {
    long long index = blockIdx.x * blockDim.x + threadIdx.x;   //blockDim.x->no of thread in the block

    if(index < audioSize){
        unsigned char audioByte = (unsigned char)audioData[index];

        outputImageData[index * 8 + 0] = (inputImageData[index * 8 + 0] | 1) & (254 + ((audioByte>>7) & 1));
        outputImageData[index * 8 + 1] = (inputImageData[index * 8 + 1] | 1) & (254 + ((audioByte>>6) & 1));
        outputImageData[index * 8 + 2] = (inputImageData[index * 8 + 2] | 1) & (254 + ((audioByte>>5) & 1));
        outputImageData[index * 8 + 3] = (inputImageData[index * 8 + 3] | 1) & (254 + ((audioByte>>4) & 1));
        outputImageData[index * 8 + 4] = (inputImageData[index * 8 + 4] | 1) & (254 + ((audioByte>>3) & 1));
        outputImageData[index * 8 + 5] = (inputImageData[index * 8 + 5] | 1) & (254 + ((audioByte>>2) & 1));
        outputImageData[index * 8 + 6] = (inputImageData[index * 8 + 6] | 1) & (254 + ((audioByte>>1) & 1));
        outputImageData[index * 8 + 7] = (inputImageData[index * 8 + 7] | 1) & (254 + ((audioByte>>0) & 1));
    }
}


int main(int argc, char *argv[]){

    char *inputImageFile = "/content/msd.ppm";
    char *inputAudioFile = "/content/a.mp3";


    // Create Cuda Events //
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
   

    // Read input image
    PPMimg *inpImg = readPPM(inputImageFile);
    int width = inpImg->width;
    int height = inpImg->height;
    long long totPixels = (long long)width * height;

    PPMpixel *inData = inpImg->data;
    PPMpixel *outData = (PPMpixel *)malloc(sizeof(PPMpixel) * totPixels);//size of one pixel * total pixels

    unsigned char * inputImageData = ppmTochar(inData, width, height); //converting arrays of pixels to linear array

    unsigned char * outputImageData = (unsigned char *)malloc(totPixels * 3ll);

    // Copy input image to device memory
    unsigned char *d_inputImageData, *d_outputImageData;
    cudaMalloc((void**)&d_inputImageData, totPixels * 3ll);
    cudaMalloc((void**)&d_outputImageData, totPixels * 3ll);

    cudaMemcpy(d_inputImageData,inputImageData,totPixels * 3ll,cudaMemcpyHostToDevice);
    cudaMemcpy(d_outputImageData,inputImageData,totPixels * 3ll,cudaMemcpyHostToDevice);    
    //--------------------------------------------------------------------------//


    // Read input audio file
    MP3File *inpAudio = readMP3(inputAudioFile);
    char *audioData = inpAudio->data;

    // Copy audio file to device memory
    char *d_audioData;
    cudaMalloc((void**)&d_audioData, inpAudio->size);
    cudaMemcpy(d_audioData, audioData, inpAudio->size, cudaMemcpyHostToDevice);
    //--------------------------------------------------------------------------//

    cout << "Size of text file = " << inpAudio->size << " bytes ("
         << (inpAudio->size * 8) << " bits)\n";
    cout << "Size of image file = " << totPixels * 3 << " bytes\n";

    // preparing invocation
    long long audioSize = inpAudio -> size; //total chunks of 1 byte read

    dim3 blockDim(THREADS_PER_BLOCK, 1, 1);
    //block and grid variables can be 1, 2, or 3 dimensional. Here (1,1) since 1D data after conversion
    dim3 gridDim((audioSize-1)/THREADS_PER_BLOCK + 1, 1, 1); //(audioSize-1)/THREADS_PER_BLOCK + 1 ->to handle case when not divisible

    cout<<"Blocks = "<<(audioSize-1)/THREADS_PER_BLOCK + 1<<"\n";

    cudaEventRecord(start);        

    //kernel invoked here
    encrypt<<<blockDim, gridDim>>>(d_inputImageData, d_outputImageData, width, height, d_audioData, audioSize);

    cudaEventRecord(stop);
    cudaEventSynchronize(stop);//Wait until the completion of all device work preceding the most recent call to cudaEventRecord()

//Since net program works asyncronusly therefore barrier is required

    float gpuTime = 0;
    cudaEventElapsedTime(&gpuTime, start, stop);
    //--------------------------------------------------------------------------//

    // Writing result to host
    cudaMemcpy(outputImageData, d_outputImageData, totPixels * 3 ,cudaMemcpyDeviceToHost);

    // Writing back output image
    char outputImageFile[] = "/content/parallel_output.ppm";
    writePPM(outputImageFile, outputImageData, inpImg->width, inpImg->height, 3);
    //--------------------------------------------------------------------------//

    // Free memory
    free(audioData);
    cudaFree(d_inputImageData);
    cudaFree(d_outputImageData);
    cudaFree(d_audioData);

    // Time Print
    cout<<"GPU Time taken (encrypt) = "<<gpuTime<<" ms\n";
}

Size of text file = 117490 bytes (939920 bits)
Size of image file = 2045424 bytes
Blocks = 230
GPU Time taken (encrypt) = 0.213504 ms



**2) Parallel Decryption ->CUDA**

In [16]:
%%writefile parallel_decrypt.cu
#include "/content/ppmHelper.h"
#include "/content/mp3Helper.h"
#include <bits/stdc++.h>

using namespace std;

#define THREADS_PER_BLOCK 512

__global__
void decrypt(unsigned char * inputImageData, int width, int height,
        char * Data, long long Size) {
    long long index = blockIdx.x * blockDim.x + threadIdx.x;

    if(index < Size){
        unsigned char Byte = 0;

        Byte |= (inputImageData[index * 8 + 0] & 1) << 7;
        Byte |= (inputImageData[index * 8 + 1] & 1) << 6;
        Byte |= (inputImageData[index * 8 + 2] & 1) << 5;
        Byte |= (inputImageData[index * 8 + 3] & 1) << 4;
        Byte |= (inputImageData[index * 8 + 4] & 1) << 3;
        Byte |= (inputImageData[index * 8 + 5] & 1) << 2;
        Byte |= (inputImageData[index * 8 + 6] & 1) << 1;
        Byte |= (inputImageData[index * 8 + 7] & 1) << 0;
        
        Data[index] = Byte;
    }
}


int main(){

    char const *inputImageFile = "/content/parallel_output (3).ppm";
    long long audioSize = 117490;
    char *outputFileExtension = ".mp3";    
    
    // Create Cuda Events //
    cudaEvent_t start, stop;
    cudaEventCreate(&start);
    cudaEventCreate(&stop);
    ////////////////////////////

    // Read input image
    PPMimg *inpImg = readPPM(inputImageFile);
    int width = inpImg->width;
    int height = inpImg->height;
    long long totPixels = (long long)width * height;

    PPMpixel *inData = inpImg->data;
    PPMpixel *outData = (PPMpixel *)malloc(sizeof(PPMpixel) * totPixels);
    unsigned char *inputImageData = ppmTochar(inData, width, height);

    // Copy input image to device memory
    unsigned char *d_inputImageData;
    cudaMalloc((void**)&d_inputImageData, totPixels * 3ll);

    cudaMemcpy(d_inputImageData,inputImageData,totPixels * 3ll,cudaMemcpyHostToDevice);
    //--------------------------------------------------------------------------//

    // Memory allocation for extracted audio
    char *extractedAudioData = (char *)malloc(audioSize);
    char *d_extractedAudioData;
    cudaMalloc((void**)&d_extractedAudioData, audioSize);
    //--------------------------------------------------------------------------//

    // Invoke Kernel
    dim3 blockDim(THREADS_PER_BLOCK, 1, 1);
    dim3 gridDim((audioSize-1)/THREADS_PER_BLOCK + 1, 1, 1);

    cudaEventRecord(start);    
    decrypt<<<blockDim, gridDim>>>(d_inputImageData, width, height, d_extractedAudioData, audioSize);
    cudaEventRecord(stop);
    cudaEventSynchronize(stop);
    float gpuTime = 0;
    cudaEventElapsedTime(&gpuTime, start, stop);
    //--------------------------------------------------------------------------//

    // Copying result to host
    cudaMemcpy(extractedAudioData, d_extractedAudioData, audioSize ,cudaMemcpyDeviceToHost);


    // Writing back audio file
    char outputAudioFile[] = "parallel_output";
    strcat(outputAudioFile, outputFileExtension);
    writeMP3(outputAudioFile, extractedAudioData, audioSize);
    //--------------------------------------------------------------------------//

    // Time Print
    cout<<"GPU Time taken (decrypt) = "<<gpuTime<<" ms\n";
}

Overwriting parallel_decrypt.cu


In [17]:
%%shell
nvcc parallel_decrypt.cu

parallel_decrypt.cu(35): warning: conversion from a string literal to "char *" is deprecated

parallel_decrypt.cu(35): warning: conversion from a string literal to "char *" is deprecated



In [18]:
%%shell
./a.out

GPU Time taken (decrypt) = 0.212864 ms


**Serial Encryption**

In [19]:
%%writefile encrypt.cpp
#include "/content/ppmHelper.h"
#include "/content/mp3Helper.h"

#include <bits/stdc++.h>

using namespace std;

int main()
{

    char const *inputImageFile = "/content/msd.ppm";
    char const *inputAudioFile = "/content/a.mp3";

    // Read input image
    PPMimg *inpImg = readPPM(inputImageFile);
    int width = inpImg->width;
    int height = inpImg->height;
    long long totPixels = (long long)width * height;

    PPMpixel *inData = inpImg->data;
    PPMpixel *outData = (PPMpixel *)malloc(sizeof(PPMpixel) * totPixels);
    unsigned char *inputImageData = ppmTochar(inData, width, height);
    unsigned char *outputImageData = (unsigned char *)malloc(totPixels * 3ll);
    //--------------------------------------------------------------------------//

    // Read input audio file
    MP3File *inpAudio = readMP3(inputAudioFile);
    char *audioData = inpAudio->data;
    //--------------------------------------------------------------------------//

    cout << "Size of  file = " << inpAudio->size << " bytes ("
         << (inpAudio->size * 8) << " bits)\n";
    cout << "Size of image file = " << totPixels * 3 << " bytes\n";

    // Check if audio file can be hidden in the given image
    if (totPixels * 3 < inpAudio->size * 8)
    {
        cout << "The image size is too small to hide the image";
        exit(0);
    }

    // Steganography
    long long imgCurrentPixel = 0;
    clock_t startTime = clock();
    for (long int i = 0; i < inpAudio->size ; i++)
    {
        unsigned char x = (unsigned char)audioData[i];
        std::bitset<8> audioByte(x);

        int bitC = 7;
        while (bitC >= 0)
        {
            outputImageData[imgCurrentPixel] = (inputImageData[imgCurrentPixel] | 1) & (254 + audioByte[bitC]);
            bitC -= 1;
            imgCurrentPixel += 1;
        }
    }
    while (imgCurrentPixel < totPixels * 3)
    {
        outputImageData[imgCurrentPixel] = inputImageData[imgCurrentPixel];
        imgCurrentPixel++;
    }
    clock_t endTime = clock();

    // Writing back output image
    char outputImageFile[] = "/content/serial_output.ppm";
    writePPM(outputImageFile, outputImageData, inpImg->width, inpImg->height, 3);
    //--------------------------------------------------------------------------//

    // Time calculation
    double cpuTime = (double)(endTime - startTime) / CLOCKS_PER_SEC;
    cout << "CPU Time taken (encrypt) = " << cpuTime * 1000 << " ms\n";

    free(audioData);
}

Overwriting encrypt.cpp


In [20]:
%%shell
g++ encrypt.cpp 

In [21]:
%%shell
./a.out

Size of  file = 117490 bytes (939920 bits)
Size of image file = 2045424 bytes
CPU Time taken (encrypt) = 31.358 ms


**Serial Decryption** 

In [22]:
%%writefile decrypt.cpp
#include "/content/ppmHelper.h"
#include "/content/mp3Helper.h"

#include <bits/stdc++.h>

using namespace std;

int main()
{

    char const *inputImageFile = "/content/serial_output.ppm";
    long long audioSize = 3351;
    char const *outputFileExtension = "txt";    

    // Read input image
    PPMimg *inpImg = readPPM(inputImageFile);
    int width = inpImg->width;
    int height = inpImg->height;
    long long totPixels = (long long)width * height;

    PPMpixel *inData = inpImg->data;
    PPMpixel *outData = (PPMpixel *)malloc(sizeof(PPMpixel) * totPixels);
    unsigned char *inputImageData = ppmTochar(inData, width, height);
    //--------------------------------------------------------------------------//

    // Steganography
    char *extractedAudioData = (char *)malloc(audioSize);
    long long imgCurrentPixel = 0;
    long long audioCurrentByte = 0;    
    
    clock_t startTime = clock();
    for (long int i = 0; i < audioSize; i++)
    {
        std::bitset<8> audioByte(0);

        int bitC = 7;
        while (bitC >= 0)
        {
            audioByte[bitC] = inputImageData[imgCurrentPixel] & 1;
            bitC -= 1;
            imgCurrentPixel += 1;
        }
        extractedAudioData[audioCurrentByte++] = static_cast<unsigned char>(audioByte.to_ulong());
    }
    clock_t endTime = clock();


    // Writing back audio file
    char outputAudioFile[] = "output.txt";
    // strcat(outputAudioFile, outputFileExtension);
    writeMP3(outputAudioFile, extractedAudioData, audioSize);
    //--------------------------------------------------------------------------//

    // Time calculation
    double cpuTime = (double)(endTime - startTime) / CLOCKS_PER_SEC;
    cout << "CPU Time taken (decrypt) = " << cpuTime * 1000 << " ms\n";
}

Writing decrypt.cpp


In [23]:
%%shell
g++ decrypt.cpp

In [25]:
%%shell
./a.out

CPU Time taken (decrypt) = 0.883 ms


# **------------------------------------------------------------------------------------------------**